In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
training=False
tuning=False


In [ ]:
if tuning or training:
    train_data =  pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
    #train_data = train_data.apply(lambda x: np.where(x.isnull(), x.dropna().sample(len(x), replace=True), x))
    train_data.fillna(train_data.mean(),inplace=True)
    
    metadata = pd.read_csv('/kaggle/input/jane-street-market-prediction/features.csv')
    metadata.drop(['feature'],axis=1,inplace=True)

    def replace_bool(tf):
        if tf:
            return 1
        else:
            return 0

    metadata = metadata.applymap(replace_bool)

    metadata_norm = metadata/metadata.sum()
    metadata_norm = metadata_norm.applymap(np.sqrt)
    features_transform = metadata_norm.values    
    
    
    start_date=86
    
    feature_columns = [col for col in train_data.columns.values if 'feature' in col]
    X_m = np.matmul(train_data[train_data['date']>=start_date][feature_columns].values,features_transform)
    
    
    
    
    corr=abs(train_data[feature_columns].corr())
    
    ordered_feature_columns=[feature_columns.pop(0)]
    
    for col in feature_columns:
        corr_max = corr[col][ordered_feature_columns].idxmax()
        corr_max_idx = ordered_feature_columns.index(corr_max)
        ordered_feature_columns.insert(corr_max_idx+1,col)
    
    f_mean = train_data[ordered_feature_columns].mean()
    f_mean.to_csv('f_mean.csv',index=False)
    import pickle
    with open("ordered_columns.txt", 'wb') as f:
        pickle.dump((ordered_feature_columns), f)
        
    resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
    
    train_data = train_data[train_data['date']>=start_date]
    X = np.concatenate((train_data[ordered_feature_columns].values,X_m),axis=1)
    y = np.stack([(train_data[c] > 0).astype('int') for c in resp_cols]).T 
    
    
    
    max_date = train_data['date'].max()
    split_date = int(0.7*(max_date-start_date))
    train_test_split = train_data['date'][train_data['date']==split_date].index[0]
    del train_data
    
    X_train=X[:train_test_split,:]
    X_test=X[train_test_split:,:]
    y_train=y[:train_test_split,:]
    y_test=y[train_test_split:,:]


In [ ]:
# corr_new=train_data[ordered_feature_columns].corr()
# import matplotlib.pyplot as plt
# f = plt.figure(figsize=(19, 15))
# plt.matshow(corr_new, fignum=f.number)
# plt.show()

In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.layers as layers
from tensorflow.keras import losses
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import kerastuner as kt
SEED=42


def create_model(hp, num_columns, num_labels):

    inp = tf.keras.layers.Input(shape = (num_columns, 1))
    #x = encoder(inp)
    #x = tf.keras.layers.Concatenate()([x,inp])
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Conv1D(filters=8,
                      kernel_size=hp.Int('kernel_size',5,10,step=5),
                      strides=1,
                      activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    for i in range(hp.Int('num_layers',12,16)): 
        x = tf.keras.layers.Dense(hp.Int(f'num_units_{i}',32,64))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(hp.Float(f'dropout_{i}',0,0.5))(x)    

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)

    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp.Float('lr',0.00001,0.1,default=0.001)),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.01), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )

    return model

In [ ]:
if tuning:   
    model_fn = lambda hp: create_model(hp,X_train.shape[-1],y_train.shape[-1])

    tuner = kt.tuners.bayesian.BayesianOptimization(
            hypermodel=model_fn,
            objective= kt.Objective('val_AUC', direction='max'),
            num_initial_points=4,
            max_trials=20)

    tuner.search(X_train,y_train,batch_size=4096,epochs=20, validation_data = (X_test, y_test), callbacks=[EarlyStopping('val_AUC', mode='max',patience=5)])
    hp  = tuner.get_best_hyperparameters(1)[0]
    pd.to_pickle(hp,'best_hp_cnn_day_86_metadata_deep.pkl')

In [ ]:
if training:
    hp = pd.read_pickle('best_hp_cnn_day_86_metadata_deep.pkl')
    model_fn = lambda hp: create_model(hp,X_train.shape[-1],y_train.shape[-1])
    model = model_fn(hp)
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=4096,
              callbacks=[EarlyStopping('val_AUC',mode='max',patience=10,restore_best_weights=True)])
    model.save_weights('JS_CNN_day_86_metadata_deep.hdf5')

In [ ]:
if not training or tuning:
    
    model_fn = lambda hp: create_model(hp,130,5)

    hp = pd.read_pickle('/kaggle/input/jscnn/best_hp_cnn_day_86.pkl')
    model = model_fn(hp)
    model.load_weights('/kaggle/input/jscnn/JS_CNN_day_86.hdf5') 
        
    samples_mean = pd.read_csv('/kaggle/input/jscnn/f_mean.csv')
    features_transform = np.load('/kaggle/input/jscnn/features_transform _130.npy')
      
    #import time
    import pickle
    import janestreet
    env = janestreet.make_env() # initialize the environment
    iter_test = env.iter_test() # an iterator which loops over the test set
#     time_of_day =-1
#     prev_date=0

    for (test_df, sample_prediction_df) in iter_test:
        weight = test_df.weight.iloc[0]
        with open("/kaggle/input/jscnn/ordered_columns.txt", 'rb') as f:
            ordered_cols = pickle.load(f)
        test_df = test_df[ordered_cols]
        X_test = test_df.values[0] 
        
        
        if weight==0:
            sample_prediction_df.action  = 0
        else:
            #X_test = X_test[1:-1]

            for index, x in np.ndenumerate(X_test):
                idx=index[0]
                if np.isnan(x):
                    X_test[idx] = samples_mean.iloc[idx]

            X_test=X_test.reshape((1,-1))
            X_metadata = np.matmul(X_test,features_transform)
            X = np.concatenate((X_test,X_metadata),axis=1)

            prediction = np.median(model(X, training=False).numpy()[0])
            #end=time.time()

            sample_prediction_df.action  = int(round(prediction))        
        env.predict(sample_prediction_df)

        #print(end-start)
        #break